<a href="https://colab.research.google.com/github/mk7890/Resume-Parsing-System/blob/main/ResumeParsingSystem_DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Resume Parsing System

The system will include text extraction, entity recognition, data cleaning, synthetic data generation, classification, CV rating, and chatbot-based CV improvement.

📌 Project Breakdown & Approach

1️⃣ Load and Process Resumes (PDF Dataset)

✅ Load resume PDFs from a dataset
✅ Extract text using pdfplumber or PyMuPDF (fitz)
✅ Save extracted text to a CSV file

2️⃣ Check and Fill Missing Values

✅ Identify missing values in extracted fields:
Applicant Name, Job Role, Email, Phone Number, Companies Worked, Work Experience, Skills, Education, Certifications, Referees
✅ Generate synthetic values for missing fields using:
Faker (for realistic names, emails, phone numbers)
Randomized industry-relevant values for missing experience, companies, skills
✅ Save the cleaned data as a new dataset

3️⃣ Generate New Resume PDFs (From Clean Data)

✅ Convert cleaned data back into a structured resume format
✅ Use FPDF or ReportLab to generate new PDFs
✅ Save new synthetic resumes in a separate folder

4️⃣ Named Entity Recognition (NER) Model for Feature Extraction

✅ Build an NLP-based Resume Parsing Model using:
spaCy (NER for extracting applicant details)
BERT / Flair for advanced entity recognition
✅ Extract all key details (Name, Job Title, Skills, Experience, Education, etc.)

5️⃣ Resume Classification (Job Role Prediction)

✅ Train a Resume Classifier
Model Choices: RandomForestClassifier, XGBoost, or BERT
Input Features: Extracted text features (TF-IDF, Word Embeddings)
Target Variable: Job roles
✅ Save the model using joblib or pickle

6️⃣ Resume Rating (Matching with Job Description)

✅ Compare extracted resume skills & experience with job descriptions
✅ Use TF-IDF + Cosine Similarity to compute a matching score
✅ Highlight missing keywords and suggest improvements

7️⃣ Chatbot for Resume Improvement

✅ Develop a chatbot using OpenAI GPT / Rasa
✅ Users can upload a job description
✅ Chatbot rewrites the resume with optimized wording & missing keywords

8️⃣ Deployment & User Interface

✅ Create a Streamlit web app to:

Upload resume PDF
Extract structured resume data
Classify job role
Rate the resume based on job description
Suggest improvements via chatbot
✅ Deploy model via Flask/FastAPI

In [ ]:
'''
📁 Resume_Parser_Project
│-- 📁 data/                 # Raw & cleaned datasets
│-- 📁 resumes/              # Original PDFs
│-- 📁 clean_resumes/        # Synthetic PDFs
│-- 📁 models/               # Trained ML models
│-- 📁 scripts/              # Python scripts
│-- 📁 app/                  # Streamlit app for deployment
│-- extract_resume_text.py   # PDF Text Extraction Script
│-- resume_parser.py         # Feature Extraction
│-- resume_classifier.py     # Job Role Prediction
│-- resume_rating.py         # CV Rating & Matching
│-- chatbot.py               # Resume Improvement Chatbot
│-- train_model.ipynb        # ML Model Training Notebook
│-- requirements.txt         # Dependencies
│-- app.py                   # Flask/FastAPI Web App
'''

# Load Libraries

Install Dependancies

In [ ]:
!pip install pdfplumber pandas faker tqdm
!pip install pymupdf pdfplumber
!pip install sentence_transformers
!pip install fpdf
!pip install sentence_transformers
!pip install faker
!pip install pymupdf pdfplumber
!pip install pdfminer.six spacy faker joblib pandas numpy scikit-learn torch transformers streamlit
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211

# Load Dataset
Load and Extract Text from PDF Resumes. I used pdfplumber for accurate extraction.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pdfplumber pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 91.3 MB/s eta 0:00:00


In [4]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=d06bf29111d31b9e7b5eaed78df4b81331ec6f99de62e26394282dc8a85b29c7
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [2]:
import gdown

# Replace 'FILE_ID' with the actual file ID from your link
file_id2 = "1vAaazotBSAS6tAZ17l6d1x7rCjzRp5te"
output_filename2 = "resume_dataset_with_features.csv"  # Change to desired output filename

# Construct the download URL
url = f"https://drive.google.com/uc?id={file_id2}"

# Download the file
gdown.download(url, output_filename2, quiet=False)

print(f"File downloaded as: {output_filename2}")

Downloading...
From (original): https://drive.google.com/uc?id=1vAaazotBSAS6tAZ17l6d1x7rCjzRp5te
From (redirected): https://drive.google.com/uc?id=1vAaazotBSAS6tAZ17l6d1x7rCjzRp5te&confirm=t&uuid=68325d27-dfe6-4040-949c-49cbd9d96808
To: /content/resume_dataset_with_features.csv
100%|██████████| 638M/638M [00:09<00:00, 66.5MB/s]

File downloaded as: resume_dataset_with_features.csv


In [5]:
import os
import pdfplumber
import pandas as pd
import re
import spacy
from tqdm import tqdm
from fpdf import FPDF

In [ ]:
# Load NLP model for better name extraction
nlp = spacy.load("en_core_web_sm")

folder_path = "/content/drive/MyDrive/resume_datasets_archive/data"
clean_folder_path = "/content/drive/MyDrive/resume_datasets_archive/clean_resumes"
os.makedirs(clean_folder_path, exist_ok=True)

In [ ]:
import os
import fitz  # PyMuPDF
import pdfplumber
import pandas as pd
from tqdm import tqdm

# Define folder path
folder_path = "/content/drive/MyDrive/resume_datasets_archive/data"
output_csv = "/content/raw_resume_data.csv"

# Function to extract text using a hybrid approach
def extract_text_from_pdf(pdf_path):
    text = ""

    # Try extracting with PyMuPDF (fitz)
    try:
        with fitz.open(pdf_path) as doc:
            text = "\n".join([page.get_text("text") for page in doc])
    except Exception as e:
        print(f"PyMuPDF failed for {pdf_path}: {e}")

    # Fallback to pdfplumber if PyMuPDF fails
    if not text.strip():
        try:
            with pdfplumber.open(pdf_path) as pdf:
                text = "\n".join([page.extract_text() for page in pdf.pages if page.extract_text()])
        except Exception as e:
            print(f"pdfplumber failed for {pdf_path}: {e}")

    return text.strip() if text else None

# List all PDF files in the folder
pdf_files = [f for f in os.listdir(folder_path) if f.endswith(".pdf")]

# Initialize list to store extracted data
resume_data = []

# Extract text from each PDF file with progress bar
for pdf_file in tqdm(pdf_files, desc="Extracting Text", unit="file"):
    pdf_path = os.path.join(folder_path, pdf_file)
    extracted_text = extract_text_from_pdf(pdf_path)
    if extracted_text:  # Ensure only non-empty text is stored
        resume_data.append({"filename": pdf_file, "text": extracted_text})

# Convert to DataFrame and save to CSV
df = pd.DataFrame(resume_data)
df.to_csv(output_csv, index=False, encoding="utf-8")

print(f"Extraction complete! Data saved to {output_csv}")


Extracting Text: 100%|██████████| 2484/2484 [01:03<00:00, 38.93file/s]


Extraction complete! Data saved to /content/raw_resume_data.csv


# Preprocessing

Check for Missing Values & Fill with Synthetic Data

✅ Extract important details like Name, Email, Phone, Skills, Job Title, Work Experience, Education, Companies Worked For, Certifications, Referees.

✅ Use regular expressions (regex) and NLP (spaCy) for extraction.

✅ Fill missing values using Faker (a Python library for generating
fake data).

In [ ]:
!pip install spacy faker
!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_lg
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 103.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 3.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 16.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load(

In [6]:
import spacy
nlp = spacy.load("en_core_web_sm")

More Feature extraction

In [ ]:
import pandas as pd
import spacy
import re
from tqdm import tqdm  # Import tqdm for progress bar

# Load spaCy NLP model (use "en_core_web_md" or "en_core_web_lg" for better entity recognition)
nlp = spacy.load("en_core_web_md")

# Load raw resume data CSV
input_csv = "/content/raw_resume_data.csv"
output_csv = "/content/parsed_resume_data2.csv"

df = pd.read_csv(input_csv)

# Define regex patterns
email_pattern = r"[a-zA-Z0-9+_.-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
phone_pattern = r"\b(\+?\d{1,3}[-.\s]?)?(\(?\d{2,4}\)?[-.\s]?)?\d{3,4}[-.\s]?\d{4}\b"
experience_pattern = r"(\d+)\s*(?:years?|yrs?)\s*(?:of)?\s*(?:experience|exp)"
linkedin_pattern = r"(https?://www\.linkedin\.com/in/[a-zA-Z0-9-_/]+)"

# Keywords for education and institutions
degree_keywords = ["bachelor", "master", "phd", "associate", "doctorate", "diploma", "certificate"]
institution_keywords = ["university", "college", "institute", "academy", "school"]

# Function to extract skills dynamically
def extract_skills(text):
    doc = nlp(text)
    skills = set()

    # Extract noun phrases that could be skills
    for chunk in doc.noun_chunks:
        if any(token.pos_ in ["NOUN", "PROPN"] for token in chunk):
            skills.add(chunk.text)

    # Look for patterns like "Proficient in Python", "Experience with SQL"
    patterns = [r"Proficient in (\w+)", r"Experience with (\w+)", r"Skilled in (\w+)"]
    for pattern in patterns:
        matches = re.findall(pattern, text, re.IGNORECASE)
        skills.update(matches)

    return ", ".join(skills) if skills else None

# Function to extract certifications dynamically
def extract_certifications(text):
    doc = nlp(text)
    certifications = set()

    for chunk in doc.noun_chunks:
        if any(keyword in chunk.text.lower() for keyword in ["certified", "certificate", "certification"]):
            certifications.add(chunk.text)

    return ", ".join(certifications) if certifications else None

# Function to extract education background
def extract_education(text):
    doc = nlp(text)
    degrees = set()
    institutions = set()

    for ent in doc.ents:
        if ent.label_ == "ORG" and any(keyword in ent.text.lower() for keyword in institution_keywords):
            institutions.add(ent.text)

    for token in doc:
        if any(keyword in token.text.lower() for keyword in degree_keywords):
            degrees.add(token.text)

    return ", ".join(degrees) if degrees else None, ", ".join(institutions) if institutions else None

# Function to extract physical address
def extract_address(text):
    doc = nlp(text)
    address = []

    for ent in doc.ents:
        if ent.label_ in ["GPE", "LOC", "FAC"]:  # Geographical entities
            address.append(ent.text)

    return ", ".join(address) if address else None

# Function to extract LinkedIn profile
def extract_linkedin(text):
    match = re.search(linkedin_pattern, text)
    return match.group() if match else None

# Function to extract structured resume data
def extract_resume_details(text):
    doc = nlp(text)

    # Extract name (first detected Proper Noun)
    name = next((ent.text for ent in doc.ents if ent.label_ == "PERSON"), None)

    # Extract job title (NER-based)
    job_title = next((ent.text for ent in doc.ents if ent.label_ in ["ORG", "WORK_OF_ART"]), None)

    # Extract email
    email = re.search(email_pattern, text)
    email = email.group() if email else None

    # Extract phone number
    phone = re.search(phone_pattern, text)
    phone = phone.group() if phone else None

    # Extract years of experience
    experience = re.search(experience_pattern, text)
    experience = experience.group(1) if experience else None

    # Extract companies worked for
    companies = [ent.text for ent in doc.ents if ent.label_ == "ORG"]
    companies = ", ".join(companies) if companies else None

    # Extract dynamic skills
    skills = extract_skills(text)

    # Extract dynamic certifications
    certifications = extract_certifications(text)

    # Extract education background and institutions
    degrees, institutions = extract_education(text)

    # Extract address
    address = extract_address(text)

    # Extract LinkedIn profile
    linkedin = extract_linkedin(text)

    # Extract referees (based on keywords like "Reference" or "Referee")
    referees = "Yes" if "reference" in text.lower() or "referee" in text.lower() else "No"

    return [name, job_title, email, phone, experience, companies, skills, certifications, degrees, institutions, address, linkedin, referees]

# Apply extraction function to dataset with progress bar
df_extracted = []
for text in tqdm(df["text"], desc="Processing Resumes", unit="resume"):
    df_extracted.append(extract_resume_details(text))

# Create structured DataFrame
df_parsed = pd.DataFrame(df_extracted, columns=[
    "Applicant Name", "Job Title", "Email", "Phone", "Years of Experience",
    "Companies Worked For", "Skills", "Certifications", "Education Background",
    "Institutions Attended", "Physical Address", "LinkedIn Profile", "Referees"
])

# Save to CSV
df_parsed.to_csv(output_csv, index=False)

print(f"Extraction complete! Processed resume data saved to {output_csv}")


Using pyresparser pretrained model

In [3]:
!pip install pyresparser nltk pandas tqdm


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 8.1 MB/s eta 0:00:00
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3960 sha256=a043d38c81702a1cde2b0042a173ca757100f9a639399cdd18db72f5647278e2
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt


In [4]:
!python -m nltk.downloader punkt
#!python -m spacy download en_core_web_sm


/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
!pip install transformers torch pandas tqdm spacy pdfplumber
#!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 63.2 MB/s eta 0:00:00
  Attempting uninstall: pdfminer.six
    Found existing installation: pdfminer.six 20240706
    Uninstalling pdfminer.six-20240706:
      Successfully uninstalled pdfminer.six-20240706


Resume Parsing with Hugging Face Transformers

In [ ]:
import pandas as pd
import re
from tqdm import tqdm
import pdfplumber
from transformers import pipeline

# Enable tqdm for pandas
tqdm.pandas()

# Load Hugging Face NER model (General-Purpose)
ner_pipeline = pipeline("ner", model="dslim/bert-base-NER", grouped_entities=True)

# Regex patterns
linkedin_pattern = r"https?://(www\.)?linkedin\.com/in/[a-zA-Z0-9-_]+"
email_pattern = r"[a-zA-Z0-9+_.-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
phone_pattern = r"\b(\+?\d{1,3}[-.\s]?)?(\(?\d{2,4}\)?[-.\s]?)?\d{3,4}[-.\s]?\d{3,4}\b"

# Keywords for different categories
education_keywords = ["Bachelor", "Master", "PhD", "BSc", "MSc", "Diploma", "Certificate", "Degree"]
certification_keywords = ["Certified", "Certification", "Credential", "Course", "Training"]
job_role_keywords = ["Engineer", "Manager", "Consultant", "Analyst", "Developer", "Scientist", "Specialist", "Technician"]
experience_keywords = ["years", "experience"]

# Load resume dataset
input_csv = "/content/raw_resume_data.csv"
output_csv = "/content/parsed_resume_data.csv"
df = pd.read_csv(input_csv)

# Function to extract named entities from text
def extract_entities(text):
    try:
        entities = ner_pipeline(text)

        # Extract structured fields
        name = next((ent["word"] for ent in entities if ent["entity_group"] == "PER"), None)
        organizations = [ent["word"] for ent in entities if ent["entity_group"] == "ORG"]
        skills = [ent["word"] for ent in entities if ent["entity_group"] == "MISC"]

        # Extract email
        email = re.search(email_pattern, text)
        email = email.group() if email else None

        # Extract phone number
        phone = re.search(phone_pattern, text)
        phone = phone.group() if phone else None

        # Extract LinkedIn profile
        linkedin = re.search(linkedin_pattern, text)
        linkedin = linkedin.group() if linkedin else None

        # Extract education background & institutions
        education = [word for word in text.split() if any(kw in word for kw in education_keywords)]
        education = ", ".join(education) if education else None

        institutions = ", ".join(organizations) if organizations else None

        # Extract job role
        job_roles = [word for word in text.split() if any(kw in word for kw in job_role_keywords)]
        job_roles = ", ".join(set(job_roles)) if job_roles else None

        # Extract certifications
        certifications = [word for word in text.split() if any(kw in word for kw in certification_keywords)]
        certifications = ", ".join(set(certifications)) if certifications else None

        # Extract years of experience
        experience_match = re.search(r"(\d+)\s*(years|year) of experience", text, re.IGNORECASE)
        years_of_experience = experience_match.group(1) if experience_match else None

        # Extract referees (basic keyword matching)
        referees = "Yes" if "reference" in text.lower() or "referee" in text.lower() else "No"

        return [
            name, job_roles, phone, email, ", ".join(organizations),
            years_of_experience, ", ".join(skills), referees, linkedin,
            certifications, education, institutions
        ]

    except Exception as e:
        print(f"Error processing resume: {e}")
        return [None] * 12  # Return empty values if an error occurs

# Apply extraction function with a progress bar
df_extracted = df["text"].progress_apply(extract_entities)

# Create structured DataFrame
df_parsed = pd.DataFrame(df_extracted.tolist(), columns=[
    "Applicant Name", "Job Role", "Phone", "Email", "Companies Worked For",
    "Years of Work Experience", "Skills", "Referees", "LinkedIn Profile",
    "Certifications", "Education Background", "Education Institutions"
])

# Save to CSV
df_parsed.to_csv(output_csv, index=False)

print(f"✅ Extraction complete! Processed resume data saved to {output_csv}")


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(
100%|██████████| 2483/2483 [02:09<00:00, 19.2

✅ Extraction complete! Processed resume data saved to /content/parsed_resume_data.csv


2️⃣ Check and Fill Missing Values


✅ Identify missing values in extracted fields: Applicant Name, Job Role, Email, Phone Number, Companies Worked, Work Experience, Skills, Education, Certifications, Referees ✅ Generate synthetic values for missing fields using: Faker (for realistic names, emails, phone numbers) Randomized industry-relevant values for missing experience, companies, skills ✅ Save the cleaned data as a new dataset

In [ ]:
parsed_resume = pd.read_csv('/content/parsed_resume_data.csv')
parsed_resume.head()

,Applicant Name,Job Role,Phone,Email,Companies Worked For,Years of Work Experience,Skills,Referees,LinkedIn Profile,Certifications,Education Background,Education Institutions
0,NaN,"Manager, Analyst",NaN,NaN,"State B, ##ANCI",NaN,"Excel, R",No,NaN,Certifications,"Master, Bachelor's","State B, ##ANCI"
1,NaN,"Technician, Engineering",NaN,NaN,"##CI, Current Engineering, HMA, ##P, ArcV, TOP...",NaN,"Improvement, Water Utility, Microsoft Access, ...",No,NaN,NaN,NaN,"##CI, Current Engineering, HMA, ##P, ArcV, TOP..."
2,NaN,Specialist,NaN,NaN,NaN,NaN,"Aid, Mass Index",No,NaN,"Certifications, Training",Diploma,NaN
3,NaN,-Manager,2008-2013,NaN,"Microsoft Office, K, Ex, NYSE, SF, NY, NY, Sec...",NaN,"NYSE Regulation, Amex, Floor, ##SE",No,NaN,"Certified, Certifications","Certificate, Certificate-Project, Certificate-LMC","Microsoft Office, K, Ex, NYSE, SF, NY, NY, Sec..."
4,Neil de Grasse Tyson,"Specialist, Manager/Supervisor",1997-2004,NaN,"MEDI, Chattanooga State, Tennessee Legislature...",NaN,"African American, Modern Times, Spanish, Spani...",No,NaN,NaN,"Degrees, Master, Bachelor","MEDI, Chattanooga State, Tennessee Legislature..."


In [ ]:
parsed_resume.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2483 entries, 0 to 2482
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Applicant Name            214 non-null    object 
 1   Job Role                  1923 non-null   object 
 2   Phone                     394 non-null    object 
 3   Email                     19 non-null     object 
 4   Companies Worked For      2334 non-null   object 
 5   Years of Work Experience  156 non-null    float64
 6   Skills                    1583 non-null   object 
 7   Referees                  2483 non-null   object 
 8   LinkedIn Profile          11 non-null     object 
 9   Certifications            1568 non-null   object 
 10  Education Background      1999 non-null   object 
 11  Education Institutions    2334 non-null   object 
dtypes: float64(1), object(11)
memory usage: 232.9+ KB


In [ ]:
parsed_resume.isnull().sum()

,0
Applicant Name,2269
Job Role,560
Phone,2089
Email,2464
Companies Worked For,149
Years of Work Experience,2327
Skills,900
Referees,0
LinkedIn Profile,2472
Certifications,915


In [ ]:
parsed_resume.columns

Index(['Applicant Name', 'Job Role', 'Phone', 'Email', 'Companies Worked For',
       'Years of Work Experience', 'Skills', 'Referees', 'LinkedIn Profile',
       'Certifications', 'Education Background', 'Education Institutions'],
      dtype='object')

In [ ]:
import pandas as pd
import re
import random
from faker import Faker

# Initialize Faker
fake = Faker()

# Input and output CSV paths
input_csv = "/content/parsed_resume_data.csv"
output_csv = "/content/cleaned_parsed_resume_data.csv"

# Load parsed resume data
df = pd.read_csv(input_csv)

# Define regex patterns for validation
email_pattern = r"[a-zA-Z0-9+_.-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
phone_pattern = r"\b(\+?\d{1,3}[-.\s]?)?(\(?\d{2,4}\)?[-.\s]?)?\d{3,4}[-.\s]?\d{4}\b"
linkedin_pattern = r"(https?:\/\/)?(www\.)?linkedin\.com\/in\/[a-zA-Z0-9-]+"

# Industry-relevant values for missing fields
random_companies = ["Google", "Microsoft", "Apple", "Amazon", "IBM", "Tesla", "Facebook", "Salesforce", "Uber", "Netflix"]
random_skills = ["Python", "SQL", "Machine Learning", "Data Analysis", "Project Management", "Cloud Computing", "Cybersecurity"]
random_experience = [str(i) for i in range(1, 21)]  # Experience between 1-20 years
random_education = ["B.Sc. Computer Science", "M.Sc. Data Science", "MBA", "Ph.D. AI", "Diploma in IT", "B.Tech Mechanical"]
random_universities = ["MIT", "Harvard", "Stanford", "Cambridge", "Oxford", "UC Berkeley", "Carnegie Mellon"]

# Function to fill missing values
def fill_missing_values(row):
    """Fill missing fields with realistic synthetic data."""
    row["Applicant Name"] = row["Applicant Name"] if pd.notna(row["Applicant Name"]) else fake.name()
    row["Email"] = row["Email"] if pd.notna(row["Email"]) and re.match(email_pattern, str(row["Email"])) else fake.email()
    row["Phone"] = row["Phone"] if pd.notna(row["Phone"]) and re.match(phone_pattern, str(row["Phone"])) else fake.phone_number()
    row["Years of Work Experience"] = row["Years of Work Experience"] if pd.notna(row["Years of Work Experience"]) else random.choice(random_experience)
    row["Companies Worked For"] = row["Companies Worked For"] if pd.notna(row["Companies Worked For"]) else random.choice(random_companies)
    row["Skills"] = row["Skills"] if pd.notna(row["Skills"]) else random.choice(random_skills)
    row["Certifications"] = row["Certifications"] if pd.notna(row["Certifications"]) else "Certified " + random.choice(random_skills)
    row["Education Background"] = row["Education Background"] if pd.notna(row["Education Background"]) else random.choice(random_education)
    row["Education Institutions"] = row["Education Institutions"] if pd.notna(row["Education Institutions"]) else random.choice(random_universities)
    row["LinkedIn Profile"] = row["LinkedIn Profile"] if pd.notna(row["LinkedIn Profile"]) and re.match(linkedin_pattern, str(row["LinkedIn Profile"])) else f"https://linkedin.com/in/{fake.user_name()}"
    row["Referees"] = row["Referees"] if pd.notna(row["Referees"]) else fake.name()

    return row

# Apply missing value filling
df_cleaned = df.apply(fill_missing_values, axis=1)

# Save cleaned data
df_cleaned.to_csv(output_csv, index=False)

print(f"Missing values filled! Cleaned resume data saved to {output_csv}")


Missing values filled! Cleaned resume data saved to /content/cleaned_parsed_resume_data.csv


Generating a new clean Resume pdf dataset

In [ ]:
resume_df = pd.read_csv('/content/cleaned_parsed_resume_data.csv')
resume_df.head()

,Applicant Name,Job Role,Phone,Email,Companies Worked For,Years of Work Experience,Skills,Referees,LinkedIn Profile,Certifications,Education Background,Education Institutions
0,Darryl Rodriguez,"Manager, Analyst",+1-810-234-8247x16972,thompsonrichard@example.com,"State B, ##ANCI",12.0,"Excel, R",No,https://linkedin.com/in/ryan73,Certifications,"Master, Bachelor's","State B, ##ANCI"
1,Allison Joseph,"Technician, Engineering",476.373.3629x1110,cpacheco@example.com,"##CI, Current Engineering, HMA, ##P, ArcV, TOP...",3.0,"Improvement, Water Utility, Microsoft Access, ...",No,https://linkedin.com/in/murphybrian,Certified Data Analysis,M.Sc. Data Science,"##CI, Current Engineering, HMA, ##P, ArcV, TOP..."
2,Christopher Williams,Specialist,294.859.3140,diana99@example.org,Amazon,9.0,"Aid, Mass Index",No,https://linkedin.com/in/barnesmary,"Certifications, Training",Diploma,UC Berkeley
3,Elizabeth Cooper,-Manager,2008-2013,johnathan90@example.com,"Microsoft Office, K, Ex, NYSE, SF, NY, NY, Sec...",13.0,"NYSE Regulation, Amex, Floor, ##SE",No,https://linkedin.com/in/areed,"Certified, Certifications","Certificate, Certificate-Project, Certificate-LMC","Microsoft Office, K, Ex, NYSE, SF, NY, NY, Sec..."
4,Neil de Grasse Tyson,"Specialist, Manager/Supervisor",1997-2004,mcgrathtina@example.net,"MEDI, Chattanooga State, Tennessee Legislature...",15.0,"African American, Modern Times, Spanish, Spani...",No,https://linkedin.com/in/xmiller,Certified Cybersecurity,"Degrees, Master, Bachelor","MEDI, Chattanooga State, Tennessee Legislature..."


In [ ]:
resume_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2483 entries, 0 to 2482
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Applicant Name            2483 non-null   object 
 1   Job Role                  1923 non-null   object 
 2   Phone                     2483 non-null   object 
 3   Email                     2483 non-null   object 
 4   Companies Worked For      2483 non-null   object 
 5   Years of Work Experience  2483 non-null   float64
 6   Skills                    2483 non-null   object 
 7   Referees                  2483 non-null   object 
 8   LinkedIn Profile          2483 non-null   object 
 9   Certifications            2483 non-null   object 
 10  Education Background      2483 non-null   object 
 11  Education Institutions    2483 non-null   object 
dtypes: float64(1), object(11)
memory usage: 232.9+ KB


In [ ]:
resume_df.isnull().sum()

,0
Applicant Name,0
Job Role,560
Phone,0
Email,0
Companies Worked For,0
Years of Work Experience,0
Skills,0
Referees,0
LinkedIn Profile,0
Certifications,0


In [ ]:
df=pd.read_csv('/content/raw_resume_data.csv')
df.head()

,filename,text
0,FINANCE (21).pdf,FINANCE AND OPERATIONS MANAGER\nExperience\nFi...
1,ENGINEERING (61).pdf,ENGINEERING TECHNICIAN\nSummary\nTo obtain a p...
2,FITNESS (24).pdf,"INTERN\nSummary\nMotivated, responsible Person..."
3,FINANCE (57).pdf,OPERATIONS FINANCE DIRECTOR\nSummary\nSkilled ...
4,DIGITAL_MEDIA (1).pdf,MEDIA ACTIVITIES SPECIALIST\nSummary\nMulti-Ta...


In [ ]:
import pandas as pd
import os

# File paths
cleaned_csv = "/content/cleaned_parsed_resume_data.csv"
raw_csv = "/content/raw_resume_data.csv"

# Load datasets
df_cleaned = pd.read_csv(cleaned_csv)
df_raw = pd.read_csv(raw_csv)

# Extract job role from the first line of text column
def extract_job_role(text):
    """Extract the first line before a newline character as the job role."""
    if pd.notna(text):
        return text.split("\n")[0].strip()
    return None

# Apply extraction to raw resume data
df_raw["Extracted Job Role"] = df_raw["text"].apply(extract_job_role)

# Convert 'filename' to match 'Applicant Name' (remove extensions)
df_raw["Applicant Name"] = df_raw["filename"].apply(lambda x: os.path.splitext(x)[0])

# Merge job roles into cleaned data using 'Applicant Name'
df_cleaned = df_cleaned.merge(df_raw[["Applicant Name", "Extracted Job Role"]], on="Applicant Name", how="left")

# Fill missing job roles
df_cleaned["Job Role"] = df_cleaned["Job Role"].fillna(df_cleaned["Extracted Job Role"])

# Drop the temporary "Extracted Job Role" column
df_cleaned.drop(columns=["Extracted Job Role"], inplace=True)

# Save the updated dataset
df_cleaned.to_csv(cleaned_csv, index=False)

print(f"✅ Missing Job Roles updated using raw resume text! Saved to {cleaned_csv}")


✅ Missing Job Roles updated using raw resume text! Saved to /content/cleaned_parsed_resume_data.csv


In [ ]:
df = pd.read_csv('/content/cleaned_parsed_resume_data.csv')
df.head()

,Applicant Name,Job Role,Phone,Email,Companies Worked For,Years of Work Experience,Skills,Referees,LinkedIn Profile,Certifications,Education Background,Education Institutions
0,Darryl Rodriguez,"Manager, Analyst",+1-810-234-8247x16972,thompsonrichard@example.com,"State B, ##ANCI",12.0,"Excel, R",No,https://linkedin.com/in/ryan73,Certifications,"Master, Bachelor's","State B, ##ANCI"
1,Allison Joseph,"Technician, Engineering",476.373.3629x1110,cpacheco@example.com,"##CI, Current Engineering, HMA, ##P, ArcV, TOP...",3.0,"Improvement, Water Utility, Microsoft Access, ...",No,https://linkedin.com/in/murphybrian,Certified Data Analysis,M.Sc. Data Science,"##CI, Current Engineering, HMA, ##P, ArcV, TOP..."
2,Christopher Williams,Specialist,294.859.3140,diana99@example.org,Amazon,9.0,"Aid, Mass Index",No,https://linkedin.com/in/barnesmary,"Certifications, Training",Diploma,UC Berkeley
3,Elizabeth Cooper,-Manager,2008-2013,johnathan90@example.com,"Microsoft Office, K, Ex, NYSE, SF, NY, NY, Sec...",13.0,"NYSE Regulation, Amex, Floor, ##SE",No,https://linkedin.com/in/areed,"Certified, Certifications","Certificate, Certificate-Project, Certificate-LMC","Microsoft Office, K, Ex, NYSE, SF, NY, NY, Sec..."
4,Neil de Grasse Tyson,"Specialist, Manager/Supervisor",1997-2004,mcgrathtina@example.net,"MEDI, Chattanooga State, Tennessee Legislature...",15.0,"African American, Modern Times, Spanish, Spani...",No,https://linkedin.com/in/xmiller,Certified Cybersecurity,"Degrees, Master, Bachelor","MEDI, Chattanooga State, Tennessee Legislature..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2483 entries, 0 to 2482
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Applicant Name            2483 non-null   object 
 1   Job Role                  1923 non-null   object 
 2   Phone                     2483 non-null   object 
 3   Email                     2483 non-null   object 
 4   Companies Worked For      2483 non-null   object 
 5   Years of Work Experience  2483 non-null   float64
 6   Skills                    2483 non-null   object 
 7   Referees                  2483 non-null   object 
 8   LinkedIn Profile          2483 non-null   object 
 9   Certifications            2483 non-null   object 
 10  Education Background      2483 non-null   object 
 11  Education Institutions    2483 non-null   object 
dtypes: float64(1), object(11)
memory usage: 232.9+ KB


In [ ]:
#fill missing values in job role column with mode and save to new csv file final_clean_resume
df['Job Role'].fillna(df['Job Role'].mode()[0], inplace=True)
df.to_csv('/content/final_clean_resume.csv', index=False)

<ipython-input-63-7cde73f172f5>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Job Role'].fillna(df['Job Role'].mode()[0], inplace=True)


## Large Resume Dataset NER Feature Extraction

In [8]:
import pandas as pd
import spacy
from spacy import displacy

In [9]:
large_resume_df = pd.read_csv('/content/resume_dataset_with_features.csv')
large_resume_df.head()

,Role,Features
0,Social Media Manager,5 to 15 Years Digital Marketing Specialist M.T...
1,Frontend Web Developer,"2 to 12 Years Web Developer BCA HTML, CSS, Jav..."
2,Quality Control Manager,0 to 12 Years Operations Manager PhD Quality c...
3,Wireless Network Engineer,4 to 11 Years Network Engineer PhD Wireless ne...
4,Conference Manager,1 to 12 Years Event Manager MBA Event planning...


In [8]:
large_resume_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1615940 entries, 0 to 1615939
Data columns (total 2 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   Role      1615940 non-null  object
 1   Features  1615940 non-null  object
dtypes: object(2)
memory usage: 24.7+ MB


In [9]:
large_resume_df.columns

Index(['Role', 'Features'], dtype='object')

In [ ]:
import pandas as pd
import re
from tqdm import tqdm
from transformers import pipeline

# Enable tqdm for pandas
tqdm.pandas()

# Load Hugging Face NER model (General-Purpose)
ner_pipeline = pipeline("ner", model="dslim/bert-base-NER", grouped_entities=True)

# Regex patterns
linkedin_pattern = r"https?://(www\.)?linkedin\.com/in/[a-zA-Z0-9-_]+"
email_pattern = r"[a-zA-Z0-9+_.-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
phone_pattern = r"\b(\+?\d{1,3}[-.\s]?)?(\(?\d{2,4}\)?[-.\s]?)?\d{3,4}[-.\s]?\d{3,4}\b"

# Keywords for different categories
education_keywords = ["Bachelor", "Master", "PhD", "BSc", "MSc", "Diploma", "Certificate", "Degree"]
certification_keywords = ["Certified", "Certification", "Credential", "Course", "Training"]
experience_keywords = ["years", "experience"]

# Load resume dataset
input_csv = "/content/resume_dataset_with_features.csv"
output_csv = "/content/parsed_resume_dataset2.csv"
df = pd.read_csv(input_csv)

# Only extract the first 150,000 rows
df = df.head(150000)

# Function to extract named entities from text
def extract_entities(row):
    try:
        text = row['Features']
        role = row['Role']
        entities = ner_pipeline(text)

        # Extract structured fields
        name = next((ent["word"] for ent in entities if ent["entity_group"] == "PER"), None)
        organizations = [ent["word"] for ent in entities if ent["entity_group"] == "ORG"]
        skills = [ent["word"] for ent in entities if ent["entity_group"] == "MISC"]

        # Extract email
        email = re.search(email_pattern, text)
        email = email.group() if email else None

        # Extract phone number
        phone = re.search(phone_pattern, text)
        phone = phone.group() if phone else None

        # Extract LinkedIn profile
        linkedin = re.search(linkedin_pattern, text)
        linkedin = linkedin.group() if linkedin else None

        # Extract education background & institutions
        education = [word for word in text.split() if any(kw in word for kw in education_keywords)]
        education = ", ".join(education) if education else None

        institutions = ", ".join(organizations) if organizations else None

        # Extract certifications
        certifications = [word for word in text.split() if any(kw in word for kw in certification_keywords)]
        certifications = ", ".join(set(certifications)) if certifications else None

        # Extract years of experience
        experience_match = re.search(r"(\d+)\s*(years|year) of experience", text, re.IGNORECASE)
        years_of_experience = experience_match.group(1) if experience_match else None

        # Extract referees (basic keyword matching)
        referees = "Yes" if "reference" in text.lower() or "referee" in text.lower() else "No"

        return [
            name, role, phone, email, ", ".join(organizations),
            years_of_experience, ", ".join(skills), referees, linkedin,
            certifications, education, institutions
        ]

    except Exception as e:
        print(f"Error processing resume: {e}")
        return [None] * 12  # Return empty values if an error occurs

# Apply extraction function with a progress bar
df_extracted = df.progress_apply(extract_entities, axis=1)

# Create structured DataFrame
df_parsed = pd.DataFrame(df_extracted.tolist(), columns=[
    "Applicant Name", "Job Role", "Phone", "Email", "Companies Worked For",
    "Years of Work Experience", "Skills", "Referees", "LinkedIn Profile",
    "Certifications", "Education Background", "Education Institutions"
])

# Save to CSV
df_parsed.to_csv(output_csv, index=False)

print(f"✅ Extraction complete! Processed resume data saved to {output_csv}")


In [10]:
# Saving unique job roles to txt file

import pandas as pd

# Load resume dataset
input_csv = "/content/resume_dataset_with_features.csv"
df = pd.read_csv(input_csv)

# Get all unique job roles
unique_job_roles = df['Role'].unique()

# Save unique job roles to a text file
output_txt = "/content/unique_job_roles.txt"
with open(output_txt, 'w') as file:
    for role in unique_job_roles:
        file.write(f"{role}\n")

print(f"✅ Unique job roles saved to {output_txt}")


✅ Unique job roles saved to /content/unique_job_roles.txt


In [16]:
# Saving unique job roles to txt file

import pandas as pd

# Load resume dataset
input_csv = "/content/final_clean_resume.csv"
df = pd.read_csv(input_csv)

# Get all unique job roles
unique_job_roles2 = df['Job Role'].unique()

# Save unique job roles to a text file
output_txt = "/content/unique_job_roles2.txt"
with open(output_txt, 'w') as file:
    for role in unique_job_roles:
        file.write(f"{role}\n")

print(f"✅ Unique job roles saved to {output_txt}")

✅ Unique job roles saved to /content/unique_job_roles2.txt


In [12]:
# Saving unique skills to txt file

import pandas as pd

# Load structured resume dataset
input_csv = "/content/structured_resume_data.csv"
df = pd.read_csv(input_csv)

# Get all unique skills
unique_skills = df['Skills'].str.split(',').explode().str.strip().unique()

# Save unique skills to a text file
output_txt = "/content/unique_skills.txt"
with open(output_txt, 'w') as file:
    for skill in unique_skills:
        file.write(f"{skill}\n")

print(f"✅ Unique skills saved to {output_txt}")


✅ Unique skills saved to /content/unique_skills.txt


In [ ]:
import pandas as pd

# Load structured resume dataset
input_csv = "/content/structured_resume_data.csv"
df = pd.read_csv(input_csv)

# Get all unique skills
unique_skills = df['Skills'].str.split(',').explode().str.strip().unique()

# Save unique skills to a text file
output_txt = "/content/unique_skills.txt"
with open(output_txt, 'w') as file:
    for skill in unique_skills:
        file.write(f"{skill}\n")

print(f"✅ Unique skills saved to {output_txt}")

In [17]:
import pandas as pd

# Load structured resume dataset
input_csv = "/content/final_clean_resume.csv"
df = pd.read_csv(input_csv)

# Get all unique skills
unique_skills = df['Skills'].str.split(',').explode().str.strip().unique()

# Save unique skills to a text file
output_txt = "/content/unique_skills2.txt"
with open(output_txt, 'w') as file:
    for skill in unique_skills:
        file.write(f"{skill}\n")

print(f"✅ Unique skills saved to {output_txt}")

✅ Unique skills saved to /content/unique_skills2.txt


# Named Entity Recognition using Hybrid Approach: spaCy and BERT

In [ ]:
import pandas as pd
import numpy as np
import spacy
from spacy import displacy
from transformers import pipeline

In [ ]:
final_clean_resume = pd.read_csv('/content/final_clean_resume.csv')
final_clean_resume.head()

,Applicant Name,Job Role,Phone,Email,Companies Worked For,Years of Work Experience,Skills,Referees,LinkedIn Profile,Certifications,Education Background,Education Institutions
0,Darryl Rodriguez,"Manager, Analyst",+1-810-234-8247x16972,thompsonrichard@example.com,"State B, ##ANCI",12.0,"Excel, R",No,https://linkedin.com/in/ryan73,Certifications,"Master, Bachelor's","State B, ##ANCI"
1,Allison Joseph,"Technician, Engineering",476.373.3629x1110,cpacheco@example.com,"##CI, Current Engineering, HMA, ##P, ArcV, TOP...",3.0,"Improvement, Water Utility, Microsoft Access, ...",No,https://linkedin.com/in/murphybrian,Certified Data Analysis,M.Sc. Data Science,"##CI, Current Engineering, HMA, ##P, ArcV, TOP..."
2,Christopher Williams,Specialist,294.859.3140,diana99@example.org,Amazon,9.0,"Aid, Mass Index",No,https://linkedin.com/in/barnesmary,"Certifications, Training",Diploma,UC Berkeley
3,Elizabeth Cooper,-Manager,2008-2013,johnathan90@example.com,"Microsoft Office, K, Ex, NYSE, SF, NY, NY, Sec...",13.0,"NYSE Regulation, Amex, Floor, ##SE",No,https://linkedin.com/in/areed,"Certified, Certifications","Certificate, Certificate-Project, Certificate-LMC","Microsoft Office, K, Ex, NYSE, SF, NY, NY, Sec..."
4,Neil de Grasse Tyson,"Specialist, Manager/Supervisor",1997-2004,mcgrathtina@example.net,"MEDI, Chattanooga State, Tennessee Legislature...",15.0,"African American, Modern Times, Spanish, Spani...",No,https://linkedin.com/in/xmiller,Certified Cybersecurity,"Degrees, Master, Bachelor","MEDI, Chattanooga State, Tennessee Legislature..."


In [ ]:
final_clean_resume.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2483 entries, 0 to 2482
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Applicant Name            2483 non-null   object 
 1   Job Role                  2483 non-null   object 
 2   Phone                     2483 non-null   object 
 3   Email                     2483 non-null   object 
 4   Companies Worked For      2483 non-null   object 
 5   Years of Work Experience  2483 non-null   float64
 6   Skills                    2483 non-null   object 
 7   Referees                  2483 non-null   object 
 8   LinkedIn Profile          2483 non-null   object 
 9   Certifications            2483 non-null   object 
 10  Education Background      2483 non-null   object 
 11  Education Institutions    2483 non-null   object 
dtypes: float64(1), object(11)
memory usage: 232.9+ KB


In [ ]:
final_clean_resume.columns

Index(['Applicant Name', 'Job Role', 'Phone', 'Email', 'Companies Worked For',
       'Years of Work Experience', 'Skills', 'Referees', 'LinkedIn Profile',
       'Certifications', 'Education Background', 'Education Institutions'],
      dtype='object')

In [ ]:
!pip install spacy transformers flair torch
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 23.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import spacy
import re
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer, pipeline
import flair
from flair.data import Sentence
from flair.models import SequenceTagger

# Load spaCy NER model
nlp_spacy = spacy.load("en_core_web_sm")

# Load BERT-based NER model (Hugging Face)
model_name = "dslim/bert-base-NER"  # Pre-trained BERT NER model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
bert_ner = pipeline("ner", model=model, tokenizer=tokenizer)

# Load Flair NER model
flair_tagger = SequenceTagger.load("flair/ner-english")

# Helper function: Extract email
def extract_email(text):
    match = re.search(r"[a-zA-Z0-9+_.-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]+", text)
    return match.group(0) if match else None

# Helper function: Extract phone number
def extract_phone(text):
    match = re.search(r"\+?\d{10,15}", text)
    return match.group(0) if match else None

# Helper function: Extract LinkedIn profile
def extract_linkedin(text):
    match = re.search(r"(https?:\/\/)?(www\.)?linkedin\.com\/[a-zA-Z0-9\-_/]+", text)
    return match.group(0) if match else None

# Extract structured data using spaCy
def extract_structured_entities(text):
    doc = nlp_spacy(text)
    name, email, phone, linkedin, certifications = None, None, None, None, []

    for ent in doc.ents:
        if ent.label_ == "PERSON":
            name = ent.text
        elif ent.label_ == "ORG":
            certifications.append(ent.text)

    email = extract_email(text)
    phone = extract_phone(text)
    linkedin = extract_linkedin(text)

    return {
        "Applicant Name": name,
        "Phone": phone,
        "Email": email,
        "LinkedIn Profile": linkedin,
        "Certifications": ", ".join(certifications) if certifications else None
    }

# Extract contextual entities using BERT
def extract_contextual_entities(text):
    job_role, skills, companies, experience = None, [], [], None

    bert_results = bert_ner(text)
    for entity in bert_results:
        entity_text = entity["word"]
        entity_type = entity["entity"].replace("B-", "").replace("I-", "")

        if entity_type == "JOB_TITLE":
            job_role = entity_text
        elif entity_type == "ORG":
            companies.append(entity_text)
        elif entity_type == "SKILL":
            skills.append(entity_text)
        elif entity_type == "DATE":  # Assume date entities might be experience
            experience = entity_text

    return {
        "Job Role": job_role,
        "Skills": ", ".join(set(skills)) if skills else None,
        "Companies Worked For": ", ".join(set(companies)) if companies else None,
        "Years of Work Experience": experience
    }

# Extract using Flair (alternative NER)
def extract_flair_entities(text):
    sentence = Sentence(text)
    flair_tagger.predict(sentence)

    job_role, education, institutions = None, [], []

    for entity in sentence.get_spans("ner"):
        if entity.tag == "ORG":
            institutions.append(entity.text)
        elif entity.tag in ["MISC", "JOB_TITLE"]:
            job_role = entity.text
        elif entity.tag == "EDUCATION":
            education.append(entity.text)

    return {
        "Education Background": ", ".join(set(education)) if education else None,
        "Education Institutions": ", ".join(set(institutions)) if institutions else None
    }

# Full resume parsing function
def parse_resume(text):
    structured_data = extract_structured_entities(text)
    contextual_data = extract_contextual_entities(text)
    flair_data = extract_flair_entities(text)

    # Merge results
    parsed_resume = {**structured_data, **contextual_data, **flair_data, "Referees": None}

    return parsed_resume

# Example usage
resume_text = """
Moses Mugambi Data Scientist Email: mugambimoses2@gmail.com | Phone: +254718695260 LinkedIn: linkedin.com/in/moses-mugambi-njeru | GitHub: github.com/mk7890
Professional Summary
Motivated Data Scientist with expertise in Python, data analysis, visualizations, and machine learning. Proficient in SQL, Tableau, Excel, and web scraping. Experienced in predictive modelling, clustering, and NLP. Passionate about deriving insights from data to solve real-world problems.
Work Experience
• Resume Parser. Built an NLP pipeline for extracting key resume details.
• Google Play Store Apps & YouTube Analysis: Data-driven insights using SQL, Pandas, and visualizations.
• Machine Learning Projects: Implemented Regression, Classification, Clustering, PCA, and CNNs.
• Time Series Analysis: Forecasted air quality trends using statistical models.
• NLP Sentiment Analysis: Analysed customer reviews for sentiment trends.
• Voice-Controlled Calculator: Developed a Python-based scientific calculator with speech recognition functionality.
Skills
• Hard Skills: Python (Pandas, NumPy, Scikit-Learn), SQL, Tableau, Machine Learning, NLP, Web Scraping, Data Visualization.
• Soft Skills: Problem-Solving, Analytical Thinking, Communication, Teamwork, Adaptability.
Education
Data Science Full - Time Zindua School, September 2024 – February 2025
B.Sc. Electrical and Electronics Engineering The Technical University of Kenya, 2013 – 2018 : Second Upper Class Honours
High School Certificate Moi High School Mbiruri, 2008 – 2011 : Grade A plain of 81 points
"""

parsed_resume = parse_resume(resume_text)
print(parsed_resume)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


pytorch_model.bin:   0%|          | 0.00/419M [00:00<?, ?B/s]

2025-02-13 23:21:58,252 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>
{'Applicant Name': 'Scikit-Learn', 'Phone': '+254718695260', 'Email': 'mugambimoses2@gmail.com', 'LinkedIn Profile': 'linkedin.com/in/moses-mugambi-njeru', 'Certifications': 'GitHub, Motivated Data Scientist, NLP, NLP, SQL, Pandas, • Machine Learning Projects, PCA, • NLP Sentiment Analysis, • Voice-Controlled, Skills\n• Hard Skills: Python, Machine Learning, NLP, Data Visualization, Analytical Thinking, Adaptability, Education, Electrical and Electronics Engineering The Technical University', 'Job Role': None, 'Skills': None, 'Companies Worked For': 'and, Kenya, Z, ##ua, Scientist, Google, Electronics, of, Technical, University, ##ind', 'Years of Work Experience': None, 'Education Background': None, 'Education Institutions': 'Technical University of Kenya, Moses Mugambi Dat